In [ ]:
#Import Libraries
import keras
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
#from keras.applications.vgg16 import preprocess_input
#from keras.preprocessing import image
#from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense, Flatten
from keras.metrics import Precision, Recall, BinaryAccuracy, FalseNegatives, FalsePositives, TrueNegatives, TruePositives
import pandas as pd
import os
import math
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint



In [ ]:
#Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_physical_devices('GPU')

In [ ]:
#Load Individual Splits
train_generator = ImageDataGenerator(rotation_range=90,
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True,
                                     validation_split=0.111,                                      
                                     vertical_flip=True,                                    
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

valid_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing
test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing

train_data_dir = 'Combined_Manual_Split\\train_combined'
test_data_dir = 'Combined_Manual_Split\\test_combined'




traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               subset='training',
                                               batch_size=32, 
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               subset='validation',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(224, 224),
                                             class_mode=None,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

In [6]:
def create_model(optimiser, fine_tune):

    #Design CNN Model
    base_model = VGG16(
    weights='imagenet',
    input_shape= (224,224,3),
    include_top=False)



    if fine_tune > 0:
        base_model.trainable = True
    else:
        base_model.trainable = False


    #Creating new Top Model

    inputs = keras.Input(shape=(224,224,3))
    top_model = base_model(inputs,training=False)
    #top_model = Flatten(name="flatten")(top_model)
    #top_model = Dense(4096, activation='relu')(top_model)
    #top_model = Dense(1072, activation='relu')(top_model)
    top_model = keras.layers.GlobalAveragePooling2D()(top_model)
    top_model = keras.layers.Dropout(0.8)(top_model)
    top_model = keras.layers.GaussianNoise(0.5)(top_model)
    output_layer = keras.layers.Dense(2,activation='sigmoid')(top_model)

    #inputs = base_model.input
    model = keras.Model(inputs,output_layer)

    model.compile(optimizer=optimiser,
            loss=keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=[keras.metrics.BinaryAccuracy()])



    #early_stopping_monitor = EarlyStopping(monitor='val_loss',patience=30,verbose=1,mode='min')

    #model_save_best = ModelCheckpoint('vgg16_combined.hdf5', save_best_only=True, monitor='val_loss', mode='min')

    return model

In [ ]:
#Run Model without fine-tuning


optimiser = keras.optimizers.Adam()
fine_tune=0


vgg_model = create_model(optimiser, fine_tune)


In [ ]:
#Train Model on data
epochs = 50
tl_checkpoint_1 = ModelCheckpoint(filepath=(os.path.join('NoFineTuned_VGG16_Models','noft_VGG16_combined_50epoch.hdf5')),
                                  monitor = 'val_loss',
                                  mode = 'min',
                                  save_best_only=True,
                                  verbose=1)

""" lr_adjust = tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=1, verbose=0, mode="auto",
    min_delta=0.0001,  cooldown=0,  min_lr=0)         
 """
hist = vgg_model.fit(traingen, epochs=epochs,batch_size=32, validation_data=validgen,callbacks=[tl_checkpoint_1])

In [ ]:

fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()
plt.savefig('noft_vgg16_combined_loss_50epoch.png')

fig = plt.figure()
plt.plot(hist.history['binary_accuracy'], color='teal', label='binary_accuracy')
plt.plot(hist.history['val_binary_accuracy'], color='orange', label='val_binary_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()
plt.savefig('noft_vgg16_combined_acc_50epoch.png')


In [ ]:
path='VGG16_Results\Epoch Tuning\Long_runs\thousandepochs'
iterations = 'r1_100epoch'
dataset = '_combined'

acc_image_name = '\VGG16_accuracy_'+iterations+dataset+'.png'
loss_image_name ='\VGG16_losses_'+iterations+dataset+'.png'
acc_string='vgg16_acc_'+iterations+dataset+'.csv'
vacc_string='vgg16_vacc_'+iterations+dataset+'.csv'
loss_string ='vgg16_loss_'+iterations+dataset+'.csv'
vloss_string='vgg16_vloss_'+iterations+dataset+'.csv'

In [ ]:
#Export model history data
model_acc = pd.DataFrame(hist.history['binary_accuracy'])
#model_acc.to_csv(acc_string,header=False,index=False)
model_acc.to_csv(acc_string,header=False,index=False)

model_val_acc = pd.DataFrame(hist.history['val_binary_accuracy'])
#model_val_acc.to_csv(vacc_string,header=False,index=False)
model_val_acc.to_csv(vacc_string,header=False,index=False)

model_loss = pd.DataFrame(hist.history['loss'])
#model_loss.to_csv(loss_string,header=False,index=False)
model_loss.to_csv(loss_string,header=False,index=False)

model_val_loss = pd.DataFrame(hist.history['val_loss'])
#model_val_loss.to_csv(vloss_string,header=False,index=False)
model_val_loss.to_csv(vloss_string,header=False,index=False)

In [ ]:

# Generate predictions
vgg_model.load_weights('NoFineTuned_VGG16_Models\\noft_VGG16_combined_50epoch.hdf5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds = vgg_model.predict(testgen)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score

vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
vgg_recall = recall_score(true_classes, vgg_pred_classes)
vgg_f1 = f1_score(true_classes, vgg_pred_classes)
vgg_precision = precision_score(true_classes, vgg_pred_classes)

print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))
print("VGG16 Model Recall without Fine-Tuning: {:.2f}%".format(vgg_recall * 100))
print("VGG16 Model Precision without Fine-Tuning: {:.2f}%".format(vgg_precision * 100))
print("VGG16 Model F1 Score without Fine-Tuning: {:.2f}%".format(vgg_f1 * 100))

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt




cm = confusion_matrix(y_true=true_classes, y_pred=vgg_pred_classes)


def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')




cm_plot_labels = ['Damage','No Damage']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')


In [ ]:
#ROC Curve
from sklearn.metrics import roc_curve

vgg_pred = vgg_model.predict(testgen).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(true_classes, vgg_pred_classes)

plt.plot(fpr_keras,tpr_keras)

Do fine-tuning on classification head

In [7]:
#Fine-Tune

# Reset our image data generators
traingen.reset()
validgen.reset()
testgen.reset()

input_shape = (224, 224, 3)

from keras.optimizers import Adam

# Use a smaller learning rate
optim_2 = Adam(learning_rate=1e-6)
fine_tune=3
# Re-compile the model, this time leaving the last 2 layers unfrozen for Fine-Tuning
vgg_model_ft = create_model(optim_2 ,fine_tune)

tl_checkpoint_1 = ModelCheckpoint(filepath=(os.path.join('Fine_Tuned_Models_VGG16','ft_vgg16_combined_100epoch.hdf5')),
                                  monitor = 'val_loss',
                                  mode = 'min',
                                  save_best_only=True,
                                  verbose=1)

""" lr_adjust = tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=1, verbose=0, mode="auto",
    min_delta=0.00001,  cooldown=0,  min_lr=0)          """                    

' lr_adjust = tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=1, verbose=0, mode="auto",\n    min_delta=0.00001,  cooldown=0,  min_lr=0)          '

In [ ]:
# Retrain model with fine-tuning
epochs =100
hist_ft = vgg_model_ft.fit(traingen, epochs=epochs,batch_size=32, validation_data=validgen,callbacks=[tl_checkpoint_1])

In [ ]:
fig = plt.figure()
plt.plot(hist_ft.history['loss'], color='teal', label='loss')
plt.plot(hist_ft.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()
plt.savefig('ft_vgg16_combined_loss_100epoch.png')

fig = plt.figure()
plt.plot(hist_ft.history['binary_accuracy'], color='teal', label='binary_accuracy')
plt.plot(hist_ft.history['val_binary_accuracy'], color='orange', label='val_binary_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()
plt.savefig('ft_vgg16_combined_acc_100epoch.png')


In [ ]:
path='VGG16_Results\Epoch Tuning\Long_runs\thousandepochs'
iterations = 'ft_100epoch'
dataset = '_combined'

acc_image_name = '\VGG16_accuracy_'+iterations+dataset+'.png'
loss_image_name ='\VGG16_losses_'+iterations+dataset+'.png'
acc_string='vgg16_acc_'+iterations+dataset+'.csv'
vacc_string='vgg16_vacc_'+iterations+dataset+'.csv'
loss_string ='vgg16_loss_'+iterations+dataset+'.csv'
vloss_string='vgg16_vloss_'+iterations+dataset+'.csv'

In [ ]:
#Export model history data
model_acc = pd.DataFrame(hist_ft.history['binary_accuracy'])
#model_acc.to_csv(acc_string,header=False,index=False)
model_acc.to_csv(acc_string,header=False,index=False)

model_val_acc = pd.DataFrame(hist_ft.history['val_binary_accuracy'])
#model_val_acc.to_csv(vacc_string,header=False,index=False)
model_val_acc.to_csv(vacc_string,header=False,index=False)

model_loss = pd.DataFrame(hist_ft.history['loss'])
#model_loss.to_csv(loss_string,header=False,index=False)
model_loss.to_csv(loss_string,header=False,index=False)

model_val_loss = pd.DataFrame(hist_ft.history['val_loss'])
#model_val_loss.to_csv(vloss_string,header=False,index=False)
model_val_loss.to_csv(vloss_string,header=False,index=False)

In [11]:
#Generate predictions on piezo set
test_data_dir_piezo = 'Combined_Manual_Split\\test_combined'

testgen = test_generator.flow_from_directory(test_data_dir_piezo,
                                             target_size=(224, 224),
                                             class_mode=None,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)




vgg_model_ft.load_weights('Fine_Tuned_Models_VGG16\\ft_vgg16_combined_100epoch.hdf5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds_ft = vgg_model_ft.predict(testgen)
vgg_pred_classes_ft = np.argmax(vgg_preds_ft, axis=1)

from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score

vgg_acc = accuracy_score(true_classes, vgg_pred_classes_ft)
vgg_recall = recall_score(true_classes, vgg_pred_classes_ft)
vgg_f1 = f1_score(true_classes, vgg_pred_classes_ft)
vgg_precision = precision_score(true_classes, vgg_pred_classes_ft)

print("VGG16 Model Accuracy with Fine-Tuning: {:.2f}%".format(vgg_acc * 100))
print("VGG16 Model Recall with Fine-Tuning: {:.2f}%".format(vgg_recall * 100))
print("VGG16 Model Precision with Fine-Tuning: {:.2f}%".format(vgg_precision * 100))
print("VGG16 Model F1 Score with Fine-Tuning: {:.2f}%".format(vgg_f1 * 100))

Found 206 images belonging to 2 classes.
206/206 [==============================] - 1s 6ms/step
VGG16 Model Accuracy with Fine-Tuning: 91.26%
VGG16 Model Recall with Fine-Tuning: 97.00%
VGG16 Model Precision with Fine-Tuning: 86.61%
VGG16 Model F1 Score with Fine-Tuning: 91.51%


In [ ]:
# Generate predictions
vgg_model_ft.load_weights('Fine_Tuned_Models_VGG16\\ft_vgg16_combined_100epoch.hdf5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds_ft = vgg_model_ft.predict(testgen)
vgg_pred_classes_ft = np.argmax(vgg_preds_ft, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score

vgg_acc = accuracy_score(true_classes, vgg_pred_classes_ft)
vgg_recall = recall_score(true_classes, vgg_pred_classes_ft)
vgg_f1 = f1_score(true_classes, vgg_pred_classes_ft)
vgg_precision = precision_score(true_classes, vgg_pred_classes_ft)

print("VGG16 Model Accuracy with Fine-Tuning: {:.2f}%".format(vgg_acc * 100))
print("VGG16 Model Recall with Fine-Tuning: {:.2f}%".format(vgg_recall * 100))
print("VGG16 Model Precision with Fine-Tuning: {:.2f}%".format(vgg_precision * 100))
print("VGG16 Model F1 Score with Fine-Tuning: {:.2f}%".format(vgg_f1 * 100))

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt




cm = confusion_matrix(y_true=true_classes, y_pred=vgg_pred_classes_ft)


def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')




cm_plot_labels = ['Damage','No Damage']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')